<a href="https://colab.research.google.com/github/gjwubyron/HateCheck/blob/main/HateCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
! git clone https://github.com/paul-rottger/hatecheck-experiments

Cloning into 'hatecheck-experiments'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 89 (delta 36), reused 38 (delta 12), pack-reused 0
Unpacking objects: 100% (89/89), done.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 42.5 MB/s 
     |████████████████████████████████| 163 kB 58.8 MB/s 


In [3]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 52 kB/s 
     |████████████████████████████████| 115 kB 11.4 MB/s 
     |████████████████████████████████| 441 kB 51.0 MB/s 
     |████████████████████████████████| 212 kB 45.6 MB/s 
     |████████████████████████████████| 95 kB 4.7 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
     |████████████████████████████████| 115 kB 66.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


# Import 


In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "bert-base-uncased"
#"textattack/bert-base-uncased-SST-2" or "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Load data

In [6]:
hatecheck_df = pd.read_csv('/content/hatecheck-experiments/Data/Test Suite/hatecheck_final_ACL.csv', index_col=0)
hatecheck_df.head()

,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ
0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
2,derog_neg_emote_h,3,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
3,derog_neg_emote_h,4,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
4,derog_neg_emote_h,5,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].


In [7]:
# Tokenize text
hatecheck_texts = hatecheck_df.test_case.astype("string").tolist()
hatecheck_labels = hatecheck_df.label_gold.replace({'hateful': 1, 'non-hateful': 0}).tolist()

encoded_data_test = tokenizer.batch_encode_plus(
    hatecheck_texts, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=64, 
    return_tensors='pt'
)


input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(hatecheck_labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test, 
                              sampler=RandomSampler(dataset_test) 
                          )

# Functions

In [9]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [10]:
from sklearn.metrics import f1_score
from prettytable import PrettyTable 

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in value_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    myTable = PrettyTable() 
    column_names = ["", label_dict_inverse[0] + "(Real)", label_dict_inverse[1] + "(Real)", "Precision"]
    myTable.add_column(column_names[0], [label_dict_inverse[0] + "(Prediction)", label_dict_inverse[1] + "(Prediction)", "Recall"])
    result = []

    for label in np.unique(labels_flat):
        
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        result.append(len(y_preds[y_preds==label]))
        result.append(len(y_true)-len(y_preds[y_preds==label]))
    myTable.add_column(column_names[1],[result[0],result[1],result[0]/(result[1]+result[0])])
    myTable.add_column(column_names[2],[result[3],result[2],result[2]/(result[3]+result[2])])
    myTable.add_column(column_names[-1],[result[0]/(result[0]+result[3]),result[2]/(result[2]+result[1]),"-"]) 
    myTable.add_column("Accuracy",[(result[0]+result[2])/sum(result),"-","-"])    
    print(myTable)

# Test

In [11]:
_, predictions, true_vals = evaluate(dataloader_test)

In [12]:
value_dict={
    'non-hateful':0,
    'hateful':1
}
accuracy_per_class(predictions, true_vals)

+-------------------------+----------------------+--------------------+--------------------+--------------------+
|                         |  Non-hateful(Real)   |   Hateful(Real)    |     Precision      |      Accuracy      |
+-------------------------+----------------------+--------------------+--------------------+--------------------+
| Non-hateful(Prediction) |          10          |         7          | 0.5882352941176471 | 0.6883047210300429 |
|   Hateful(Prediction)   |         1155         |        2556        | 0.6887631366208569 |         -          |
|          Recall         | 0.008583690987124463 | 0.9972688255950058 |         -          |         -          |
+-------------------------+----------------------+--------------------+--------------------+--------------------+
